In [1]:
from ctapipe.core import Tool
from traitlets import Dict, List, Integer
from traitlets.config.loader import KVArgParseConfigLoader
from traitlets.config.application import catch_config_error
from ipython_genutils import py3compat
from copy import deepcopy, copy

from ctapipe.io.files import InputFile
from ctapipe.io.hessio import hessio_event_source
from ctapipe.utils.datasets import get_path
from ctapipe.io import CameraGeometry
from ctapipe.calib.camera.new_tool_proposal import FactoryTool
import numpy as np

from ctapipe.calib.camera.charge_extraction import ChargeExtractorFactory

In [2]:
def get_test_parameters():
    parameters = {"integrator": "nb_peak_integration",
                  "integration_window": [7, 3],
                  "integration_sigamp": [2, 4],
                  "integration_lwt": 0}
    return parameters


def get_test_event():
    filename = get_path('gamma_test.simtel.gz')
    for event in hessio_event_source(filename):
        if event.dl0.event_id == 409:
            return event

telid = 11
event = get_test_event()
nsamples = event.dl0.tel[telid].num_samples
data = np.array(list(event.dl0.tel[telid].adc_samples.values()))
ped = event.dl0.tel[telid].pedestal
data_ped = data - np.atleast_3d(ped / nsamples)
data_ped = np.array([data_ped[0], data_ped[0]])

pixel_pos = event.meta.pixel_pos[telid]
optical_foclen = event.meta.optical_foclen[telid]

geom = CameraGeometry.guess(*pixel_pos, optical_foclen)
nei = geom.neighbors

params = get_test_parameters()

In [3]:
class CalTool(FactoryTool):
    name = "mytool"
    description = "do some things and stuff"
    
    factories = Dict(dict(extractor=ChargeExtractorFactory))
    classes = List([])
    aliases = Dict(dict(extractor='ChargeExtractorFactory.extractor'))

    def setup(self):
        self.extractor_factory = self.factories['extractor'](config=self.config)
        self.extractor = self.extractor_factory.get_product(data_ped, nei=[[2]], config=self.config)
        pass
        
    def start(self):
        print(self.classes)
        print(self.extractor.window_width)
        print(self.extractor.extract_charge())
        #a = self.extractor(data_ped, None, config=self.config)
        #print(a.window_width)
        #print(self.products[0].window_width)

    def finish(self):
        print("fin")
        self.log.warning("Shutting down.")

In [4]:
t = CalTool()

In [5]:
argv = ['--extractor','LocalPeakIntegrator', '--window_width', '10']
t.run(argv)

    INFO [CalTool]: version 0.1.dev1027 [release=False] [githash=36793a8cede4ba19b49db4455f5aa089a121dc32]
    INFO [CalTool]: Starting: mytool
 WARNING [CalTool]: Shutting down.


here
here
[<class '__main__.CalTool'>, <class 'ctapipe.calib.camera.charge_extraction.ChargeExtractorFactory'>, <class 'ctapipe.calib.camera.charge_extraction.LocalPeakIntegrator'>]
10
(masked_array(data =
 [[42 59 45 26 94 56 -39 39 156 -38 77 29 13 90 85 34 72 3 106 -31 108 90 94
  -30 123 89 0 59 140 67 27 46 27 23 -50 126 158 94 12 164 -33 -12 37 153
  98 95 8 40 -12 75 20 93 -61 59 16 -59 -19 -43 98 271 -37 26 87 140 23 104
  3 55 98 114 87 85 36 40 -6 104 108 202 56 45 111 1 138 18 77 125 2 19 23
  108 0 70 95 126 79 43 14 29 49 73]
 [42 59 45 26 94 56 -39 39 156 -38 77 29 13 90 85 34 72 3 106 -31 108 90 94
  -30 123 89 0 59 140 67 27 46 27 23 -50 126 158 94 12 164 -33 -12 37 153
  98 95 8 40 -12 75 20 93 -61 59 16 -59 -19 -43 98 271 -37 26 87 140 23 104
  3 55 98 114 87 85 36 40 -6 104 108 202 56 45 111 1 138 18 77 125 2 19 23
  108 0 70 95 126 79 43 14 29 49 73]],
             mask =
 [[False False False ..., False False False]
 [False False False ..., False False False]],
    

In [6]:
argv = ['--extractor','LocalPeakIntegrator', '--window_width', '10', '--help']
t.run(argv)

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--extractor=<Unicode> (ChargeExtractorFactory.extractor)
    Default: 'NeighbourPeakIntegrator'
    Charge extraction scheme to use: ['Integrator', 'FullIntegrator',
    'WindowIntegrator', 'SimpleIntegrator', 'PeakFindingIntegrator',
    'GlobalPeakIntegrator', 'LocalPeakIntegrator', 'NeighbourPeakIntegrator']
--sig_amp_cut_LG=<Int> (LocalPeakIntegrator.sig_amp_cut_LG)
    Default: 4
    Define the cut above which a sample is considered as significant for
    PeakFinding in the LG channel
--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--window_width=<Int> (LocalPeakIntegrator.window_width)
    Default: 7
    Define the width

SystemExit: 0

/Users/Jason/anaconda3/envs/cta/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
